In [1]:
import os
import cv2
import sys
import random

import json
import h5py
import itertools
import numpy as np
from PIL import Image
import argparse, pickle

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from tqdm import tqdm
from torch import optim
import torchvision.models
from torch.utils.data import Dataset as torchDataset
from torch.utils.data import DataLoader as TorchDataLoader
from torch.utils.data.dataloader import default_collate

# caption libraries
import evaluation
import collections
from data.example import Example
from data.utils import nostdout
from data.field import ImageDetectionsField, TextField, RawField
from models.transformer import Transformer, MemoryAugmentedEncoder, MeshedDecoder, ScaledDotProductAttentionMemory

# graph libraries
import utils.io as io
import matplotlib.pyplot as plt
from utils.g_vis_img import *
from models.graph_su import *
from evaluation.graph_eval import *

# feature extractor
from models.feature_extractor import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Random seeds
seed = 27
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Dataset Constants

In [2]:
class SurgicalSceneConstants():
    '''
    Surgical Scene constants
    '''
    def __init__( self):
        self.instrument_classes = ('kidney', 'bipolar_forceps', 'prograsp_forceps', 'large_needle_driver',
                                'monopolar_curved_scissors', 'ultrasound_probe', 'suction', 'clip_applier',
                                'stapler', 'maryland_dissector', 'spatulated_monopolar_cautery')
        self.action_classes = ( 'Idle', 'Grasping', 'Retraction', 'Tissue_Manipulation', 
                                'Tool_Manipulation', 'Cutting', 'Cauterization', 
                                'Suction', 'Looping', 'Suturing', 'Clipping', 'Staple', 
                                'Ultrasound_Sensing')

# Cross-entropy loss with label smoothing

In [3]:
class CELossWithLS(torch.nn.Module):
    '''
    label smoothing cross-entropy loss for captioning
    '''
    def __init__(self, classes=None, smoothing=0.1, gamma=3.0, isCos=True, ignore_index=-1):
        super(CELossWithLS, self).__init__()
        self.complement = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.log_softmax = torch.nn.LogSoftmax(dim=1)
        self.gamma = gamma
        self.ignore_index = ignore_index

    def forward(self, logits, target):
        with torch.no_grad():
            oh_labels = F.one_hot(target.to(torch.int64), num_classes = self.cls).permute(0,1,2).contiguous()
            smoothen_ohlabel = oh_labels * self.complement + self.smoothing / self.cls

        logs = self.log_softmax(logits[target!=self.ignore_index])
        pt = torch.exp(logs)
        return -torch.sum((1-pt).pow(self.gamma)*logs * smoothen_ohlabel[target!=self.ignore_index], dim=1).mean()


# Dataloader

In [4]:
class DataLoader(TorchDataLoader):
    def __init__(self, dataset, *args, **kwargs):
        super(DataLoader, self).__init__(dataset, *args, collate_fn=dataset.collate_fn(), **kwargs)

class Dataset(object):
    def __init__(self, examples, fields, gsu_const):
        self.examples = examples
        self.fields = dict(fields)
        
        self.file_dir = gsu_const['file_dir']
        self.img_dir = gsu_const['img_dir']
        self.dataconst = gsu_const['dataconst']
        self.feature_extractor = gsu_const['feature_extractor']
        self.word2vec = h5py.File(gsu_const['w2v_loc'], 'r')
        
    # word2vec
    def _get_word2vec(self,node_ids):
        word2vec = np.empty((0,300))
        for node_id in node_ids:
            vec = self.word2vec[self.dataconst.instrument_classes[node_id]]
            word2vec = np.vstack((word2vec, vec))
        return word2vec

    def __getitem__(self, i):
        example = self.examples[i]
        frame_path = getattr(example, 'image')
        frame_path = frame_path.split("/")
        
        _img_loc = os.path.join(self.file_dir, frame_path[0],self.img_dir,frame_path[3].split("_")[0]+'.png')
        frame_data = h5py.File(os.path.join(self.file_dir, frame_path[0],'vsgat',self.feature_extractor, frame_path[3].split("_")[0]+'_features.hdf5'), 'r')    
        
        #print(_img_loc)
        
        # caption data
        cp_data = []
        for field_name, field in self.fields.items():
            if field_name == 'image' and field == None:
                cp_data.append(np.zeros((6,512), dtype = np.float32))
            else:
                cp_data.append(field.preprocess(getattr(example, field_name)))   
        if len(cp_data) == 1: cp_data = cp_data[0]
        
        # graph data
        gsu_data = {}
        gsu_data['img_name'] = frame_data['img_name'].value[:] + '.jpg'
        gsu_data['img_loc'] = _img_loc
        gsu_data['node_num'] = frame_data['node_num'].value
        gsu_data['roi_labels'] = frame_data['classes'][:]
        gsu_data['det_boxes'] = frame_data['boxes'][:]
        gsu_data['edge_labels'] = frame_data['edge_labels'][:]
        gsu_data['edge_num'] = gsu_data['edge_labels'].shape[0]
        if self.fields['image'] == None:
            gsu_data['features'] = np.zeros((gsu_data['node_num'],512), dtype = np.float32)
        else:
            gsu_data['features'] = frame_data['node_features'][:]
        gsu_data['spatial_feat'] = frame_data['spatial_features'][:]
        gsu_data['word2vec'] = self._get_word2vec(gsu_data['roi_labels'])
        
        data = {}
        data['cp_data'] = cp_data
        data['gsu_data'] = gsu_data
        return data

    def __len__(self):
        return len(self.examples)

    def __getattr__(self, attr):
        if attr in self.fields:
            for x in self.examples:
                yield getattr(x, attr)
                
    def collate_fn(self):
        def collate(batch):
            gsu_batch_data = {}
            gsu_batch_data['img_name'] = []
            gsu_batch_data['img_loc'] = []
            gsu_batch_data['node_num'] = []
            gsu_batch_data['roi_labels'] = []
            gsu_batch_data['det_boxes'] = []
            gsu_batch_data['edge_labels'] = []
            gsu_batch_data['edge_num'] = []
            gsu_batch_data['features'] = []
            gsu_batch_data['spatial_feat'] = []
            gsu_batch_data['word2vec'] = []

            for data in batch:
                gsu_batch_data['img_name'].append(data['gsu_data']['img_name'])
                gsu_batch_data['img_loc'].append(data['gsu_data']['img_loc'])
                gsu_batch_data['node_num'].append(data['gsu_data']['node_num'])
                gsu_batch_data['roi_labels'].append(data['gsu_data']['roi_labels'])
                gsu_batch_data['det_boxes'].append(data['gsu_data']['det_boxes'])
                gsu_batch_data['edge_labels'].append(data['gsu_data']['edge_labels'])
                gsu_batch_data['edge_num'].append(data['gsu_data']['edge_num'])
                gsu_batch_data['features'].append(data['gsu_data']['features'])
                gsu_batch_data['spatial_feat'].append(data['gsu_data']['spatial_feat'])
                gsu_batch_data['word2vec'].append(data['gsu_data']['word2vec'])

            gsu_batch_data['edge_labels'] = torch.FloatTensor(np.concatenate(gsu_batch_data['edge_labels'], axis=0))
            gsu_batch_data['features'] = torch.FloatTensor(np.concatenate(gsu_batch_data['features'], axis=0))
            gsu_batch_data['spatial_feat'] = torch.FloatTensor(np.concatenate(gsu_batch_data['spatial_feat'], axis=0))
            gsu_batch_data['word2vec'] = torch.FloatTensor(np.concatenate(gsu_batch_data['word2vec'], axis=0))
            
            cp_batch_data = []
            tensors = []
            
            for data in batch: cp_batch_data.append(data['cp_data'])
            if len(self.fields) == 1: cp_batch_data = [cp_batch_data, ]
            else: cp_batch_data = list(zip(*cp_batch_data))

            for field, data in zip(self.fields.values(), cp_batch_data):
                if field == None: tensor = default_collate(data)
                else: tensor = field.process(data)
                if isinstance(tensor, collections.Sequence) and any(isinstance(t, torch.Tensor) for t in tensor):
                    tensors.extend(tensor)
                else: tensors.append(tensor)

            if len(tensors) > 1:cp_batch_data = tensors
            else: cp_batch_data = tensors[0]
            
            batch_data = {}
            batch_data['gsu'] = gsu_batch_data
            batch_data['cp'] = cp_batch_data
            
            return(batch_data)

        return collate
    
class PairedDataset(Dataset):
    def __init__(self, examples, fields, gsu_const):
        assert ('image' in fields)
        assert ('text' in fields)
        super(PairedDataset, self).__init__(examples, fields, gsu_const)
        self.image_field = self.fields['image']
        if self.image_field == None: print('no pre-extracted image featured')
        self.text_field = self.fields['text']
        
    def image_dictionary(self, fields=None):
        if not fields:
            fields = self.fields
        dataset = Dataset(self.examples, fields, gsu_const)
        #dataset = DictionaryDataset(self.examples, fields, gsu_const, 'image')
        return dataset
        
class COCO(PairedDataset):
    def __init__(self, image_field, text_field, gsu_const, img_root, ann_root, id_root=None):
        # setting training and val root
        roots = {}
        roots['train'] = { 'img': img_root, 'cap': os.path.join(ann_root, 'captions_train.json')}
        roots['val'] = {'img': img_root, 'cap': os.path.join(ann_root, 'captions_val.json')}

        # Getting the id: planning to remove this in future
        if id_root is not None:
            ids = {}
            ids['train'] = json.load(open(os.path.join(id_root, 'WithCaption_id_path_train.json'), 'r'))
            ids['val'] = json.load(open(os.path.join(id_root, 'WithCaption_id_path_val.json'), 'r'))   
        else: ids = None
        
        with nostdout():
            self.train_examples, self.val_examples = self.get_samples(roots, ids)
        examples = self.train_examples + self.val_examples
        super(COCO, self).__init__(examples, {'image': image_field, 'text': text_field}, gsu_const)   

    @property
    def splits(self):
        train_split = PairedDataset(self.train_examples, self.fields, gsu_const) 
        val_split = PairedDataset(self.val_examples, self.fields, gsu_const)
        return train_split, val_split

    @classmethod
    def get_samples(cls, roots, ids_dataset=None):
        train_samples = []
        val_samples = []
   
        for split in ['train', 'val']:
            anns = json.load(open(roots[split]['cap'], 'r'))
            if ids_dataset is not None: ids = ids_dataset[split]
                
            for index in range(len(ids)):              
                id_path = ids[index]
                caption = anns[index]['caption']
                example = Example.fromdict({'image': os.path.join('', id_path), 'text': caption})
                if split == 'train': train_samples.append(example)
                elif split == 'val': val_samples.append(example)
                    
        return train_samples, val_samples

# Arguments, dataloader

In [5]:
# arguments
device = torch.device('cuda')
parser = argparse.ArgumentParser(description='Incremental domain adaptation for surgical report generation')
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--workers', type=int, default=0)

# caption
parser.add_argument('--exp_name', type=str, default='m2_transformer')
parser.add_argument('--m', type=int, default=40)   
parser.add_argument('--cp_cbs', type=str, default='True')
parser.add_argument('--cp_cbs_filter', default='LOG', type=str) # Potential choice: 'gau' and 'LOG'
parser.add_argument('--cp_kernel_sizex', default=3, type=int)
parser.add_argument('--cp_kernel_sizey', default=1, type=int)
parser.add_argument('--cp_decay_epoch', default=2, type=int) 
parser.add_argument('--cp_std_factor', default=0.9, type=float)

# graph
parser.add_argument('--gsu_cbs',        type=bool, default=True)
parser.add_argument('--gsu_feat', type=str,  default='resnet18_09_SC_CBS')
parser.add_argument('--gsu_w2v_loc', type=str,  default='datasets/surgicalscene_word2vec.hdf5')

# feature_extractor
parser.add_argument('--fe_use_cbs',            type=bool,      default=True,        help='use CBS')
parser.add_argument('--fe_std',                type=float,     default=1.0,         help='')
parser.add_argument('--fe_std_factor',         type=float,     default=0.9,         help='')
parser.add_argument('--fe_cbs_epoch',          type=int,       default=5,           help='')
parser.add_argument('--fe_kernel_size',        type=int,       default=3,           help='')
parser.add_argument('--fe_fil1',               type=str,       default='LOG',       help='gau, LOG')
parser.add_argument('--fe_fil2',               type=str,       default='gau',       help='gau, LOG')
parser.add_argument('--fe_fil3',               type=str,       default='gau',       help='gau, LOG')
parser.add_argument('--fe_num_classes',        type=int,       default=11,           help='11')
parser.add_argument('--fe_use_SC',             type=bool,      default=True,       help='use SuperCon')

# checkpoints and file dirs
print('Training check for DA_ECBS_ResNet18_09_SC_ECBS')

parser.add_argument('--gsu_img_dir', type=str,  default='left_frames')
parser.add_argument('--gsu_file_dir', type=str,  default='datasets/instruments18/')

parser.add_argument('--cp_features_path', type=str, default='datasets/instruments18/') 
parser.add_argument('--cp_annotation_folder', type=str, default='datasets/annotations_new/annotations_SD_inc')

# checkpoints

args = parser.parse_args(args=[])
print(args)

# graph scene understanding constants
gsu_const = {}
gsu_const['file_dir'] = args.gsu_file_dir
gsu_const['img_dir'] = args.gsu_img_dir
gsu_const['dataconst'] = SurgicalSceneConstants()
gsu_const['feature_extractor'] = args.gsu_feat
gsu_const['w2v_loc'] =args.gsu_w2v_loc


# Pipeline for image regions and text
#image_field = ImageDetectionsField(detections_path=args.cp_features_path, max_detections=6, load_in_tmp=False)  
image_field = None
text_field = TextField(init_token='<bos>', eos_token='<eos>', lower=True, tokenize='spacy', remove_punctuation=True, nopoints=False)

# Create the dataset 
dataset = COCO(image_field, text_field, gsu_const, args.cp_features_path, args.cp_annotation_folder, args.cp_annotation_folder)
train_dataset, val_dataset = dataset.splits   
print('train:', len(train_dataset))
print('val:', len(val_dataset))
    
# caption data
if not os.path.isfile('datasets/vocab_%s.pkl' % args.exp_name):
    print("Building vocabulary")
    text_field.build_vocab(train_dataset, val_dataset, min_freq=2)  
    pickle.dump(text_field.vocab, open('datasets/vocab_%s.pkl' % args.exp_name, 'wb'))
else:
    text_field.vocab = pickle.load(open('datasets/vocab_%s.pkl' % args.exp_name, 'rb'))

print('vocabulary size is:', len(text_field.vocab))
print(text_field.vocab.stoi)

# dataset
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers)
dict_dataset_val = val_dataset.image_dictionary({'image': image_field, 'text': RawField()})
dict_dataloader_val = DataLoader(dict_dataset_val, batch_size=args.batch_size) # for caption with word GT class number

Training check for DA_ECBS_ResNet18_09_SC_ECBS
Namespace(batch_size=32, cp_annotation_folder='datasets/annotations_new/annotations_SD_inc', cp_cbs='True', cp_cbs_filter='LOG', cp_decay_epoch=2, cp_features_path='datasets/instruments18/', cp_kernel_sizex=3, cp_kernel_sizey=1, cp_std_factor=0.9, exp_name='m2_transformer', fe_cbs_epoch=5, fe_fil1='LOG', fe_fil2='gau', fe_fil3='gau', fe_kernel_size=3, fe_num_classes=11, fe_std=1.0, fe_std_factor=0.9, fe_use_SC=True, fe_use_cbs=True, gsu_cbs=True, gsu_feat='resnet18_09_SC_CBS', gsu_file_dir='datasets/instruments18/', gsu_img_dir='left_frames', gsu_w2v_loc='datasets/surgicalscene_word2vec.hdf5', m=40, workers=0)
no pre-extracted image featured
no pre-extracted image featured
no pre-extracted image featured
train: 1560
val: 447
vocabulary size is: 41
defaultdict(<function _default_unk_index at 0x7f68d8ba08c8>, {'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3, 'is': 4, 'tissue': 5, 'forceps': 6, 'monopolar': 7, 'curved': 8, 'scissors': 9, 'bipo

# Feature Extractor

In [6]:
# net model
if args.fe_use_SC: feature_network = SupConResNet(args=args)
else: feature_network = ResNet18(args)

# CBS
if args.fe_use_cbs:
    if args.fe_use_SC: feature_network.encoder.get_new_kernels(0)
    else: feature_network.get_new_kernels(0)

# gpu
num_gpu = torch.cuda.device_count()
if num_gpu > 0:
    device_ids = np.arange(num_gpu).tolist()    
    if args.fe_use_SC:
        feature_network.encoder = torch.nn.DataParallel(feature_network.encoder)
        feature_network = feature_network.cuda()
    else:
        feature_network = nn.DataParallel(feature_network, device_ids=device_ids).cuda()

# Caption model

In [7]:
if args.cp_cbs == 'True':
    from models.transformer import MemoryAugmentedEncoder_CBS
    print("MemoryAugmentedEncoder_CBS")
    encoder = MemoryAugmentedEncoder_CBS(3, 0, attention_module=ScaledDotProductAttentionMemory, attention_module_kwargs={'m': args.m})
else:
    print("MemoryAugmentedEncoder")
    encoder = MemoryAugmentedEncoder(3, 0, attention_module=ScaledDotProductAttentionMemory, attention_module_kwargs={'m': args.m}) 

decoder = MeshedDecoder(len(text_field.vocab), 54, 3, text_field.vocab.stoi['<pad>'])
caption_model = Transformer(text_field.vocab.stoi['<bos>'], encoder, decoder).to(device)

if args.cp_cbs == 'True':
    caption_model.encoder.get_new_kernels(0, args.cp_kernel_sizex, args.cp_kernel_sizey, args.cp_decay_epoch, args.cp_std_factor, args.cp_cbs_filter) 

MemoryAugmentedEncoder_CBS


# Graph Model

In [8]:
graph_su_model = AGRNN(bias= True, bn=False, dropout=0.3, multi_attn=False, layer=1, diff_edge=False, use_cbs = args.gsu_cbs)
if args.gsu_cbs:
    graph_su_model.grnn1.gnn.apply_h_h_edge.get_new_kernels(0)

# Load Pre-trained_weights independent weights

In [9]:
# fe_modelpath ='feature_extractor/checkpoint/incremental/inc_ResNet18_SC_CBS_0_012345678.pkl'
# gsu_checkpoint = 'checkpoints/g_checkpoints/da_ecbs_resnet18_09_SC_eCBS/da_ecbs_resnet18_09_SC_eCBS/epoch_train/checkpoint_D1230_epoch.pth'
# cp_checkpoint = 'checkpoints/IDA_MICCAI2021_checkpoints/SD_base_LOG/'

# # feature network
# feature_network.load_state_dict(torch.load(args.fe_modelpath))

# # caption
# pretrained_model = torch.load(args.cp_checkpoint+('%s_best.pth' % args.exp_name))
# caption_model.load_state_dict(pretrained_model['state_dict']) 

# # graph
# pretrained_model = torch.load(args.gsu_checkpoint)
# graph_su_model.load_state_dict(pretrained_model['state_dict'])

# feature extraction layers

In [10]:
# extract the encoder layer
if args.fe_use_SC:
    feature_network = feature_network.encoder
else:
    if args.fe_use_cbs: feature_network = nn.Sequential(*list(feature_network.module.children())[:-2])
    else: feature_network = nn.Sequential(*list(feature_network.module.children())[:-1])

feature_network = feature_network.cuda()

# MTL Model (Graph Scene Understanding and Captioning)

In [11]:
class mtl_model(nn.Module):
    '''
    Multi-task model : Graph Scene Understanding and Captioning
    '''
    def __init__(self, feature_extractor, graph, caption):
        super(mtl_model, self).__init__()
        self.feature_extractor = feature_extractor
        self.graph_su = graph
        self.caption = caption
        self.transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    
    def forward(self, img_dir, det_boxes_all, caps_gt, node_num, features, spatial_feat, word2vec, roi_labels, val = False, text_field = None):               
        
        gsu_node_feat = None
        cp_node_feat = None
        for index, img_loc in  enumerate(img_dir):
            #print(img_loc)
            _img = Image.open(img_loc).convert('RGB')
            _img = np.array(_img)
            
            img_stack = None
            for idx, bndbox in enumerate(det_boxes_all[index]):        
                roi = np.array(bndbox).astype(int)
                roi_image = _img[roi[1]:roi[3] + 1, roi[0]:roi[2] + 1, :]
                roi_image = self.transform(cv2.resize(roi_image, (224, 224), interpolation=cv2.INTER_LINEAR))
                roi_image = torch.autograd.Variable(roi_image.unsqueeze(0))
                # stack nodes images per image
                if img_stack is None: img_stack = roi_image
                else: img_stack = torch.cat((img_stack, roi_image))
            
            # send the stack to feature extractor
            img_stack = img_stack.cuda()
            feature = feature_network(img_stack)
            feature = feature.view(feature.size(0), -1)        
            
            if gsu_node_feat == None: gsu_node_feat = feature
            else: gsu_node_feat = torch.cat((gsu_node_feat,feature))
            
            feature = torch.unsqueeze(torch.cat((feature,torch.zeros((6-len(feature)),512).cuda())),0)
            if cp_node_feat == None: cp_node_feat = feature
            else: cp_node_feat = torch.cat((cp_node_feat,feature))
                
        #print('feat_size',gsu_node_feat.shape)
        #print('cp_feat', cp_node_feat.shape)
    
        if val == True:
            caption_output, _ = self.caption.beam_search(cp_node_feat, 20, text_field.vocab.stoi['<eos>'], 5, out_size=1)
        else:
            caption_output = self.caption(cp_node_feat, caps_gt)
        interaction = self.graph_su(node_num, gsu_node_feat, spatial_feat, word2vec, roi_labels)
        #print(interaction)
        #print(interaction.shape)
        return interaction, caption_output

# Combined_model

In [12]:
model = mtl_model(feature_network, graph_su_model, caption_model)
model = model.to(device)

# Evaluation

In [13]:
import itertools


def eval_mtl(epoch, model, dataloader, text_field):
    
    
    #model.eval()
    gen = {}
    gts = {}

    # graph
    # criterion and scheduler
    g_criterion = nn.MultiLabelSoftMarginLoss()                   
    g_edge_count = 0
    g_total_acc = 0.0
    g_total_loss = 0.0
    g_logits_list = []
    g_labels_list = []
    
    for it, data in tqdm(enumerate(iter(dataloader))):
            
        graph_data = data['gsu']
        cp_data = data['cp']
            
        # graph
        img_name = graph_data['img_name']
        img_loc = graph_data['img_loc']
        node_num = graph_data['node_num']
        roi_labels = graph_data['roi_labels']
        det_boxes = graph_data['det_boxes']
        edge_labels = graph_data['edge_labels']
        edge_num = graph_data['edge_num']
        features = graph_data['features']
        spatial_feat = graph_data['spatial_feat']
        word2vec = graph_data['word2vec']
        features, spatial_feat, word2vec, edge_labels = features.to(device), spatial_feat.to(device), word2vec.to(device), edge_labels.to(device)         
        
        # caption
        _, caps_gt = cp_data
            
        with torch.no_grad():
    
            g_output, caption_out = model(img_loc, det_boxes, caps_gt, node_num, features, spatial_feat, word2vec, roi_labels, val = True, text_field = text_field)
        
            g_logits_list.append(g_output)
            g_labels_list.append(edge_labels)
            # loss and accuracy
            g_loss = g_criterion(g_output, edge_labels.float())
            g_acc = np.sum(np.equal(np.argmax(g_output.cpu().data.numpy(), axis=-1), np.argmax(edge_labels.cpu().data.numpy(), axis=-1)))
            
        # accumulate loss and accuracy of the batch
        g_total_loss += g_loss.item() * edge_labels.shape[0]
        g_total_acc  += g_acc
        g_edge_count += edge_labels.shape[0]
        
        caps_gen = text_field.decode(caption_out, join_words=False)
        
        for i, (gts_i, gen_i) in enumerate(zip(caps_gt, caps_gen)):
            gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
            gen['%d_%d' % (it, i)] = [gen_i, ]    
            gts['%d_%d' % (it, i)] = [gts_i,]
        
    #graph evaluation
    g_total_acc = g_total_acc / g_edge_count
    g_total_loss = g_total_loss / len(dataloader)

    g_logits_all = torch.cat(g_logits_list).cuda()
    g_labels_all = torch.cat(g_labels_list).cuda()
    g_logits_all = F.softmax(g_logits_all, dim=1)
    g_map_value, g_ece, g_sce, g_tace, g_brier, g_uce = calibration_metrics(g_logits_all, g_labels_all, 'test')
    
    # caption evaluation
    #if not os.path.exists('results/c_results/predict_caption'):
    #    os.makedirs('results/c_results/predict_caption')
    #json.dump(gen, open('results/c_results/predict_caption/predict_caption_val.json', 'w'))

    gts = evaluation.PTBTokenizer.tokenize(gts)
    gen = evaluation.PTBTokenizer.tokenize(gen)

    scores, _ = evaluation.compute_scores(gts, gen)
    print('Graph : {acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f}' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce.item()) )
    print(print("Caption Scores :", scores))
    return #scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce.item()

# Load pretrained MTL model

In [17]:
mtl_checkpoint = 'checkpoints/mtl_train/UDA_Graph/checkpoint_'
    
for i in range(1,101):
    # MTL
    pretrained_model = torch.load(mtl_checkpoint+str(i)+'_epoch.pth')
    model.load_state_dict(pretrained_model['state_dict'])
    print('epoch:',i)
    eval_mtl(i, model, dict_dataloader_val, text_field)

0it [00:00, ?it/s]

epoch: 1


14it [00:54,  3.91s/it]
0it [00:00, ?it/s]

Graph : {acc: 0.476314 map: 0.167071 loss: 28.179157, ece:0.137035, sce:0.069644, tace:0.067284, brier:0.774015, uce:0.078060}
Caption Scores : {'BLEU': array([0.5122, 0.4437, 0.3825, 0.3289]), 'METEOR': 0.3125, 'ROUGE': 0.4989, 'CIDEr': 1.9248}
None
epoch: 2


14it [00:59,  4.24s/it]
0it [00:00, ?it/s]

...

Graph : {acc: 0.475452 map: 0.155219 loss: 28.790698, ece:0.131780, sce:0.067217, tace:0.064464, brier:0.781218, uce:0.080023}
Caption Scores : {'BLEU': array([0.506 , 0.4399, 0.3798, 0.3262]), 'METEOR': 0.3094, 'ROUGE': 0.497, 'CIDEr': 2.0055}
None
epoch: 99


14it [00:59,  4.24s/it]
0it [00:00, ?it/s]

Graph : {acc: 0.488372 map: 0.158030 loss: 28.781845, ece:0.145541, sce:0.066064, tace:0.063622, brier:0.773853, uce:0.094215}
Caption Scores : {'BLEU': array([0.5081, 0.4429, 0.3835, 0.3296]), 'METEOR': 0.3122, 'ROUGE': 0.4957, 'CIDEr': 2.088}
None
epoch: 100


14it [00:59,  4.26s/it]


Graph : {acc: 0.470284 map: 0.154727 loss: 28.153866, ece:0.127032, sce:0.066442, tace:0.064375, brier:0.776224, uce:0.072268}
Caption Scores : {'BLEU': array([0.5002, 0.4343, 0.3742, 0.3201]), 'METEOR': 0.3073, 'ROUGE': 0.4888, 'CIDEr': 1.8817}
None


In [17]:
mtl_checkpoint = 'checkpoints/mtl_train/UDA/checkpoint_'

pretrained_model = torch.load(mtl_checkpoint+str(1)+'_epoch.pth')
model.load_state_dict(pretrained_model['state_dict'])
print('epoch:',1)
eval_mtl(1, model, dict_dataloader_val, text_field)
#eval_mtl(1, model, train_dataloader, text_field)

0it [00:00, ?it/s]

epoch: 1


1it [00:00,  2.08it/s]

torch.Size([5, 13])


2it [00:00,  2.12it/s]

...

torch.Size([7, 13])


8it [00:03,  2.24it/s]

torch.Size([6, 13])


8it [00:03,  2.02it/s]


KeyboardInterrupt: 

# Need to work on evaluation and TS/CDA-TS

In [ ]:
''' 1.0 mutlitask model base evaluation ==========================================================================='''
scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce = evaluate_metrics(model, device, dict_dataloader_val, text_field)
print('Initial Graph SU: acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce))
print("Initial Caption scores :", scores)

# Evaluation Matrix : Graph (Loss, Acc, ECE), Caption (Brier, Cider)

In [6]:
def evaluate_metrics(model, device, dataloader, text_field, g_temp = 1.5, c_temp = None):
    import itertools
    
    model.caption.decoder.caption_ts = c_temp
    
    model.eval()
    gen = {}
    gts = {}

    # graph
    # criterion and scheduler
    g_criterion = nn.MultiLabelSoftMarginLoss()                   
    g_edge_count = 0
    g_total_acc = 0.0
    g_total_loss = 0.0
    g_logits_list = []
    g_labels_list = []

    #print(model.caption.beam_search)
    
    with tqdm(desc='evaluation', unit='it', total=len(dataloader)) as pbar:
        for it, (images, caps_gt, graph_data) in enumerate(iter(dataloader)):
            
            # graph
            img_name = graph_data['img_name']
            img_loc = graph_data['img_loc']
            node_num = graph_data['node_num']
            roi_labels = graph_data['roi_labels']
            det_boxes = graph_data['det_boxes']
            edge_labels = graph_data['edge_labels']
            edge_num = graph_data['edge_num']
            features = graph_data['features']
            spatial_feat = graph_data['spatial_feat']
            word2vec = graph_data['word2vec']
            features, spatial_feat, word2vec, edge_labels = features.to(device), spatial_feat.to(device), word2vec.to(device), edge_labels.to(device)    
        
            # caption
            images = images.to(device)

            with torch.no_grad():
                # caption
                caption_out, _ = model.caption.beam_search(images, 20, text_field.vocab.stoi['<eos>'], 5, out_size=1)
                #print(caption_out)
                # graph
                g_output = model.graph_su(node_num, features, spatial_feat, word2vec, roi_labels, validation=True)
                g_output = g_output/g_temp
                g_logits_list.append(g_output)
                g_labels_list.append(edge_labels)
                # loss and accuracy
                g_loss = g_criterion(g_output, edge_labels.float())
                g_acc = np.sum(np.equal(np.argmax(g_output.cpu().data.numpy(), axis=-1), np.argmax(edge_labels.cpu().data.numpy(), axis=-1)))
            
            # accumulate loss and accuracy of the batch
            g_total_loss += g_loss.item() * edge_labels.shape[0]
            g_total_acc  += g_acc
            g_edge_count += edge_labels.shape[0]
            
            caps_gen = text_field.decode(caption_out, join_words=False)
            for i, (gts_i, gen_i) in enumerate(zip(caps_gt, caps_gen)):
                gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
                gen['%d_%d' % (it, i)] = [gen_i, ]    
                gts['%d_%d' % (it, i)] = gts_i
            pbar.update()
    
    #graph loss
    g_logits_all = torch.cat(g_logits_list).cuda()
    g_labels_all = torch.cat(g_labels_list).cuda()
    g_total_acc = g_total_acc / g_edge_count
    g_total_loss = g_total_loss / len(dataloader)

    g_logits_all = F.softmax(g_logits_all, dim=1)
    g_map_value, g_ece, g_sce, g_tace, g_brier, g_uce = calibration_metrics(g_logits_all, g_labels_all, 'test')
    print('acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce.item()) )


    if not os.path.exists('results/c_results/predict_caption'):
        os.makedirs('results/c_results/predict_caption')
    json.dump(gen, open('results/c_results/predict_caption/predict_caption_val.json', 'w'))

    gts = evaluation.PTBTokenizer.tokenize(gts)
    gen = evaluation.PTBTokenizer.tokenize(gen)
    scores, _ = evaluation.compute_scores(gts, gen)
    return scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce.item()

# Temperature Scaling : Model Object

In [8]:
class ModelWithTemperature(nn.Module):
    '''
    Temperature scaling model for model
    '''
    def __init__(self, model):
        super(ModelWithTemperature, self).__init__()
        self.model = model
        self.use_ts = False
        self.graph_su_temperature = nn.Parameter(torch.ones(1) * 1.5)
        self.caption_temperature = nn.Parameter(torch.ones(1) * 1.5)

    def forward(self, detections, captions, batch_graph, batch_h_node_list, batch_obj_node_list, batch_h_h_e_list, batch_o_o_e_list, batch_h_o_e_list, feat, spatial_feat, word2vec):
        
        # if self.use_ts:
            # g_su_temp = self.graph_su_temperature.unsqueeze(1).expand(interaction.size(0), interaction.size(1))
            # self.model.caption.decoder.caption_ts = self.caption_temperature
        
        caption_output, interaction = self.model(detections, captions, batch_graph, batch_h_node_list, batch_obj_node_list, batch_h_h_e_list, batch_o_o_e_list, batch_h_o_e_list, feat, spatial_feat, word2vec)
        
        # if self.use_ts:
            # interaction = interaction / g_su_temp

        return caption_output, interaction

    def graph_su_set_temperature(self, valid_loader):
        '''
        Finding optimal temperature scale for graph scene understanding task
        '''
        self.cuda()
        g_logits_list = []
        g_labels_list = []
        g_criterion = nn.MultiLabelSoftMarginLoss()
        
        with torch.no_grad():
            for it, (images, caps_gt, graph_data) in enumerate(iter(valid_loader)):
                # graph
                img_name = graph_data['img_name']
                img_loc = graph_data['img_loc']
                node_num = graph_data['node_num']
                roi_labels = graph_data['roi_labels']
                det_boxes = graph_data['det_boxes']
                edge_labels = graph_data['edge_labels']
                edge_num = graph_data['edge_num']
                features = graph_data['features']
                spatial_feat = graph_data['spatial_feat']
                word2vec = graph_data['word2vec']
                features, spatial_feat, word2vec, edge_labels = features.to(device), spatial_feat.to(device), word2vec.to(device), edge_labels.to(device)    
                
                #caption_out = self.model.caption(images, caps_gt)
                g_output = self.model.graph_su(node_num, features, spatial_feat, word2vec, roi_labels, validation=True)
                    
                g_logits_list.append(g_output)
                g_labels_list.append(edge_labels)
                
            g_logits = torch.cat(g_logits_list).cuda()
            g_labels = torch.cat(g_labels_list).cuda()

        #init_temp = self.graph_su_temperature.clone()
        optimizer = optim.LBFGS([self.graph_su_temperature], lr=0.01, max_iter=50)

        def eval():
            g_su_temp = self.graph_su_temperature.unsqueeze(1).expand(g_logits.size(0), g_logits.size(1))
            g_logit_out = F.softmax(g_logits/g_su_temp, dim=1)
            g_loss = g_criterion(g_logit_out, g_labels.float())
            g_loss.backward()
            return g_loss
        
        optimizer.step(eval)
        return

    def caption_set_temperature(self, valid_loader):
        '''
        Finding optimal temperature scale for caption task
        '''
        self.cuda()
        c_logits_list = None
        c_labels_list = None
        
        with torch.no_grad():
            for it, (images, caps_gt) in enumerate(iter(valid_loader)):    
                images, caps_gt = images.to(device), caps_gt.to(device)
                caption_out = self.model.caption(images, caps_gt)

                if c_logits_list is not None:
                    c_logits_list = torch.cat([c_logits_list, caption_out], 1)
                    c_labels_list = torch.cat([c_labels_list, caps_gt],1)
                else:
                    c_logits_list = caption_out
                    c_labels_list = caps_gt

            c_logits = c_logits_list.cuda()
            c_labels = c_labels_list.cuda()
        
        init_temp = self.caption_temperature.clone()
        optimizer = optim.LBFGS([self.caption_temperature], lr=0.01, max_iter=50)

        def eval():
            caption_temp = self.caption_temperature.unsqueeze(1).expand(c_logits.size(1), c_logits.size(0))
            c_criterion = CELossWithLS(classes=len(text_field.vocab), smoothing=0.1, gamma=0.0, isCos=False, ignore_index=text_field.vocab.stoi['<pad>'])
            c_base = c_logits/caption_temp
            c_loss = c_criterion(c_base[:, :-1].contiguous(), c_labels[:, 1:].contiguous())
            c_loss.backward()
            return c_loss
        
        optimizer.step(eval)
        return

# Temperature Scaling : ECE Loss

In [9]:
class _ECELoss(nn.Module):
    '''
    Expected Calibration Error for Calibration
    '''
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(_ECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        if labels.size(1) == 13:
            gt_labels = torch.argmax(labels, dim=1)
        else: 
            logits = logits.squeeze()
            gt_labels = labels.squeeze()

        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(gt_labels)
        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
        return ece

# Temperature Scaling : Evaluation

In [10]:
def ts_evaluation(model_ts, g_c_val_dataloader, c_val_dataloader, g_temp=1.5, c_temp=1):
    '''
    '''
    model.eval()

    g_acc = 0.0
    c_acc = 0.0
    g_loss = 0.0
    c_loss = 0.0
    g_logits = []
    g_labels = []
    c_logits = []
    c_labels = []
    
    g_logits_list = []
    g_labels_list = []
    c_logits_list = None
    c_labels_list = None

    with torch.no_grad():
        for it, (images, caps_gt, graph_data) in enumerate(iter(g_c_val_dataloader)):
            # graph
            img_name = graph_data['img_name']
            img_loc = graph_data['img_loc']
            node_num = graph_data['node_num']
            roi_labels = graph_data['roi_labels']
            det_boxes = graph_data['det_boxes']
            edge_labels = graph_data['edge_labels']
            edge_num = graph_data['edge_num']
            features = graph_data['features']
            spatial_feat = graph_data['spatial_feat']
            word2vec = graph_data['word2vec']
            features, spatial_feat, word2vec, edge_labels = features.to(device), spatial_feat.to(device), word2vec.to(device), edge_labels.to(device)    
        
            g_output = model_ts.model.graph_su(node_num, features, spatial_feat, word2vec, roi_labels, validation=True)
            
            g_output = g_output / g_temp
            g_logits_list.append(g_output)
            g_labels_list.append(edge_labels)
                
    g_logits = torch.cat(g_logits_list).cuda()
    g_labels = torch.cat(g_labels_list).cuda()

    g_logits = F.softmax(g_logits, dim=1)
    g_criterion = nn.MultiLabelSoftMarginLoss()
    g_loss = g_criterion(g_logits, g_labels.float())
    g_acc = np.sum(np.equal(np.argmax(g_logits.cpu().data.numpy(), axis=-1), np.argmax(g_labels.cpu().data.numpy(), axis=-1))) / g_labels.size(0)

    with torch.no_grad():
        for it, (images, caps_gt) in enumerate(iter(c_val_dataloader)):    
            images, caps_gt = images.to(device), caps_gt.to(device)
            caption_out = model_ts.model.caption(images, caps_gt)
            caption_out = caption_out/c_temp
            if c_logits_list is not None:
                c_logits_list = torch.cat([c_logits_list, caption_out], 1)
                c_labels_list = torch.cat([c_labels_list, caps_gt],1)
            else:
                c_logits_list = caption_out
                c_labels_list = caps_gt

    c_logits = c_logits_list.cuda()
    c_labels = c_labels_list.cuda()
    
    c_criterion = CELossWithLS(classes=len(text_field.vocab), smoothing=0.1, gamma=0.0, isCos=False, ignore_index=text_field.vocab.stoi['<pad>'])
    c_loss = c_criterion(c_logits[:, :-1].contiguous(), c_labels[:, 1:].contiguous())
    c_acc = np.sum(np.equal(np.argmax(c_logits.cpu().data.numpy(), axis=-1), c_labels.cpu().data.numpy())) / c_labels.size(1)
    
    return (g_loss.item(), c_loss.item(), g_acc, c_acc, g_logits, g_labels, c_logits, c_labels)

# 2.0: Temperature Scaling

In [11]:
model_ts = ModelWithTemperature(model)
ece_criterion = _ECELoss().to(device)
print('Initial Graph SU Temperature:%.4f'%model_ts.graph_su_temperature.item())
print('Initial Caption Temperature:%0.4f'%model_ts.caption_temperature.item())

Initial Graph SU Temperature:1.5000
Initial Caption Temperature:1.5000


# 2.1: Temperature Scaling : Find Optimal value

In [12]:
model_ts.graph_su_set_temperature(dict_dataloader_val)
model_ts.caption_set_temperature(dataloader_val)
print('-----------------------------------------------------------------------')
print('Optimal Graph SU Temperature:%.4f'%model_ts.graph_su_temperature.item())
print('Optimal Caption Temperature:%0.4f'%model_ts.caption_temperature.item())

-----------------------------------------------------------------------
Optimal Graph SU Temperature:1.3954
Optimal Caption Temperature:4.8063


# 2.2 Temperature Scaling : Without TS

In [13]:
g_loss, c_loss, g_acc, c_acc, g_logits, g_labels, c_logits, c_labels = ts_evaluation(model_ts, dict_dataloader_val, 
                                    dataloader_val)
g_temperature_ece = ece_criterion(g_logits, g_labels).item()
c_temperature_ece = ece_criterion(c_logits, c_labels).item()
print('-----------------------------------------------------------------------')
print('Before TS: G_loss:%.3f, G_acc:%.3f, G_ece:%.5f'%(g_loss, g_acc, g_temperature_ece))
print('Before TS: C_loss:%.3f, C_acc:%.3f, C_ece:%.5f'%(c_loss, c_acc, c_temperature_ece))

scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce = evaluate_metrics(model_ts.model, device, dict_dataloader_val, text_field)
print('Before TS: Graph SU: acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce))
print("Before TS: Caption scores :", scores)

evaluation:   0%|          | 0/447 [00:00<?, ?it/s]

-----------------------------------------------------------------------
Before TS: G_loss:0.710, G_acc:0.571, G_ece:0.46277
Before TS: C_loss:2.724, C_acc:0.022, C_ece:0.84285


evaluation: 100%|██████████| 447/447 [01:56<00:00,  3.85it/s]


acc: 0.571059 map: 0.321686 loss: 0.509024, ece:0.219497, sce:0.047773, tace:0.049699, brier:0.649657, uce:0.283210
Before TS: Graph SU: acc: 0.571059 map: 0.321686 loss: 0.509024, ece:0.219497, sce:0.047773, tace:0.049699, brier:0.649657, uce:0.283210
Before TS: Caption scores : {'BLEU': array([0.5498, 0.4714, 0.4238, 0.3801]), 'METEOR': 0.2861, 'ROUGE': 0.57, 'CIDEr': 2.7487}


# 2.3 Temperature Scaling : With TS

In [14]:
g_loss, c_loss, g_acc, c_acc, g_logits, g_labels, c_logits, c_labels = ts_evaluation(model_ts, dict_dataloader_val, 
                                    dataloader_val, g_temp=model_ts.graph_su_temperature.item(), c_temp=model_ts.caption_temperature.item())
g_temperature_ece = ece_criterion(g_logits, g_labels).item()
c_temperature_ece = ece_criterion(c_logits, c_labels).item()
print('-----------------------------------------------------------------------')
print('Optimal TS: G_loss:%.3f, G_acc:%.3f, G_ece:%.5f'%(g_loss, g_acc, g_temperature_ece))
print('Optimal TS: C_loss:%.3f, C_acc:%.3f, C_ece:%.5f'%(c_loss, c_acc, c_temperature_ece))

scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce = evaluate_metrics(model_ts.model, device, dict_dataloader_val, text_field, g_temp = model_ts.graph_su_temperature.item(), c_temp = model_ts.caption_temperature.item())
print('Optimal TS: Graph SU: acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce))
print("Optimal TS: Caption scores :", scores)

evaluation:   0%|          | 0/447 [00:00<?, ?it/s]

-----------------------------------------------------------------------
Optimal TS: G_loss:0.709, G_acc:0.571, G_ece:0.46025
Optimal TS: C_loss:1.894, C_acc:0.022, C_ece:0.29004


evaluation: 100%|██████████| 447/447 [01:57<00:00,  3.80it/s]


acc: 0.571059 map: 0.315094 loss: 0.487805, ece:0.218120, sce:0.045566, tace:0.046718, brier:0.644169, uce:0.260813
Optimal TS: Graph SU: acc: 0.571059 map: 0.315094 loss: 0.487805, ece:0.218120, sce:0.045566, tace:0.046718, brier:0.644169, uce:0.260813
Optimal TS: Caption scores : {'BLEU': array([0.3623, 0.3231, 0.3032, 0.2864]), 'METEOR': 0.2151, 'ROUGE': 0.4628, 'CIDEr': 2.3378}


# 2.4 Temperature Scaling : With CDA-TS

In [15]:
g_cls_freq = torch.zeros(13)
for i in torch.argmax(g_labels, dim=1): g_cls_freq[i] += 1
g_cls_freq_norm = g_cls_freq/torch.max(g_cls_freq)
g_temp = model_ts.graph_su_temperature.item() + g_cls_freq_norm*0.1
#g_temp = g_temp.to(device)
model_ts.graph_su_temperature = nn.Parameter(g_temp.to(device))

c_cls_freq = torch.zeros(41)
for i in c_labels.squeeze(): c_cls_freq[i] += 1
c_cls_freq_norm = c_cls_freq/torch.max(c_cls_freq)
c_temp = model_ts.caption_temperature.item() + c_cls_freq_norm*0.1
#c_temp = c_temp.to(device)
model_ts.caption_temperature = nn.Parameter(c_temp.to(device))

g_loss, c_loss, g_acc, c_acc, g_logits, g_labels, c_logits, c_labels = ts_evaluation(model_ts, dict_dataloader_val, 
                                    dataloader_val, g_temp = model_ts.graph_su_temperature, c_temp = model_ts.caption_temperature)
g_temperature_ece = ece_criterion(g_logits, g_labels).item()
c_temperature_ece = ece_criterion(c_logits, c_labels).item()
print('-----------------------------------------------------------------------')
print('CDA-TS: G_loss:%.3f, G_acc:%.3f, G_ece:%.5f'%(g_loss, g_acc, g_temperature_ece))
print('CDA-TS: C_loss:%.3f, C_acc:%.3f, C_ece:%.5f'%(c_loss, c_acc, c_temperature_ece))

scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce = evaluate_metrics(model_ts.model, device, dict_dataloader_val, text_field, g_temp = model_ts.graph_su_temperature, c_temp = model_ts.caption_temperature)
print('CDA-TS: Graph SU: acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce))
print("CDA-TS: Caption scores :", scores)

evaluation:   0%|          | 0/447 [00:00<?, ?it/s]

-----------------------------------------------------------------------
CDA-TS: G_loss:0.709, G_acc:0.574, G_ece:0.46348
CDA-TS: C_loss:3.262, C_acc:0.022, C_ece:0.02099


evaluation: 100%|██████████| 447/447 [01:56<00:00,  3.85it/s]


acc: 0.573643 map: 0.319211 loss: 0.489725, ece:0.217880, sce:0.045660, tace:0.046676, brier:0.643576, uce:0.267471
CDA-TS: Graph SU: acc: 0.573643 map: 0.319211 loss: 0.489725, ece:0.217880, sce:0.045660, tace:0.046676, brier:0.643576, uce:0.267471
CDA-TS: Caption scores : {'BLEU': array([0.3623, 0.3231, 0.3032, 0.2864]), 'METEOR': 0.2151, 'ROUGE': 0.4628, 'CIDEr': 2.3378}


# Confidence Aware distribution

In [16]:
def conf_dist(model_ts, g_c_val_dataloader, c_val_dataloader):

    g_conf_list = np.zeros(13)
    c_conf_list = np.zeros(41)

    with torch.no_grad():
        for it, (images, caps_gt, graph_data) in enumerate(iter(g_c_val_dataloader)):
            # graph
            img_name = graph_data['img_name']
            img_loc = graph_data['img_loc']
            node_num = graph_data['node_num']
            roi_labels = graph_data['roi_labels']
            det_boxes = graph_data['det_boxes']
            edge_labels = graph_data['edge_labels']
            edge_num = graph_data['edge_num']
            features = graph_data['features']
            spatial_feat = graph_data['spatial_feat']
            word2vec = graph_data['word2vec']
            features, spatial_feat, word2vec, edge_labels = features.to(device), spatial_feat.to(device), word2vec.to(device), edge_labels.to(device)    
        
            g_output = model_ts.model.graph_su(node_num, features, spatial_feat, word2vec, roi_labels, validation=True)
            
            g_output = F.softmax(g_output,1)
            g_confidences, g_predictions = torch.max(g_output, 1)
            g_accuracies = g_predictions.eq(torch.argmax(edge_labels, dim=1))
            for i in torch.argmax(edge_labels, dim=1).unique():
                g_conf_list[i] += g_confidences[torch.argmax(edge_labels, dim=1)==i].sum()

    
    with torch.no_grad():
        for it, (images, caps_gt) in enumerate(iter(c_val_dataloader)):    
            images, caps_gt = images.to(device), caps_gt.to(device)
            c_output = model_ts.model.caption(images, caps_gt)
            c_output = c_output.squeeze()
            c_output = F.softmax(c_output,1)
            c_confidences, c_predictions = torch.max(c_output, 1)
            c_accuracies = c_predictions.eq(caps_gt.squeeze())
            for i in caps_gt.squeeze().unique():
                c_conf_list[i] += c_confidences[caps_gt.squeeze()==i].sum()

    return g_conf_list, c_conf_list

# 2.5 Temperature Scaling : With CCA-TS

In [17]:
g_conf_list, c_conf_list = conf_dist(model_ts, dict_dataloader_val, dataloader_val)

plt.figure(1)
plt.title('Graph SU class distribution')
plt.bar(np.arange(len(g_cls_freq)),g_cls_freq)
plt.savefig('graph_class_dist.png')

plt.figure(2)
plt.title('Graph SU confidence score distribution')
plt.bar(np.arange(len(g_cls_freq)),g_conf_list/12)
plt.savefig('graph_conf_dist.png')
    
plt.figure(3)
plt.title('Caption Class Distribution')
plt.bar(np.arange(len(c_cls_freq)),c_cls_freq)
plt.savefig('caption_cls_dist.png')
    
plt.figure(4)
plt.title('Caption Confidence distribution')
plt.bar(np.arange(len(c_cls_freq)),c_conf_list/41)
plt.savefig('caption_conf_dist.png')

# CCA-TS calculation V2

In [18]:
g_cls_freq = g_conf_list/13
g_cls_freq_norm = g_cls_freq/np.max(g_cls_freq)
g_cls_freq_norm = torch.tensor(g_cls_freq_norm).float()
g_temp = model_ts.graph_su_temperature.cpu() + g_cls_freq_norm*0.1
# g_temp = g_temp.to(device)
model_ts.graph_su_temperature = nn.Parameter(g_temp.to(device))
    

c_cls_freq = c_conf_list/41#train_dataset.get_cls_num_list()
c_cls_freq_norm = c_cls_freq/np.max(c_cls_freq)
c_cls_freq_norm = torch.tensor(c_cls_freq_norm).float()
c_temp =  model_ts.caption_temperature.cpu() + c_cls_freq_norm*0.1
# c_temp = c_temp.to(device)
model_ts.caption_temperature = nn.Parameter(c_temp.to(device))
    
g_loss, c_loss, g_acc, c_acc, g_logits, g_labels, c_logits, c_labels = ts_evaluation(model_ts, dict_dataloader_val, 
                                    dataloader_val, g_temp = model_ts.graph_su_temperature, c_temp = model_ts.caption_temperature)
g_temperature_ece = ece_criterion(g_logits, g_labels).item()
c_temperature_ece = ece_criterion(c_logits, c_labels).item()
print('-----------------------------------------------------------------------')
print('CCA-TS-V1: G_loss:%.3f, G_acc:%.3f, G_ece:%.5f'%(g_loss, g_acc, g_temperature_ece))
print('CCA-TS-V1: C_loss:%.3f, C_acc:%.3f, C_ece:%.5f'%(c_loss, c_acc, c_temperature_ece))

scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce = evaluate_metrics(model_ts.model, device, dict_dataloader_val, text_field, g_temp = model_ts.graph_su_temperature, c_temp = model_ts.caption_temperature)
print('CCA-TS-V1: Graph SU: acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce))
print("CCA-TS-V1: Caption scores :", scores)

evaluation:   0%|          | 0/447 [00:00<?, ?it/s]

-----------------------------------------------------------------------
CCA-TS-V1: G_loss:0.709, G_acc:0.575, G_ece:0.46582
CCA-TS-V1: C_loss:3.272, C_acc:0.022, C_ece:0.02043


evaluation: 100%|██████████| 447/447 [01:56<00:00,  3.85it/s]


acc: 0.575366 map: 0.320200 loss: 0.491720, ece:0.216305, sce:0.045280, tace:0.046805, brier:0.643167, uce:0.266481
CCA-TS-V1: Graph SU: acc: 0.575366 map: 0.320200 loss: 0.491720, ece:0.216305, sce:0.045280, tace:0.046805, brier:0.643167, uce:0.266481
CCA-TS-V1: Caption scores : {'BLEU': array([0.3623, 0.3231, 0.3032, 0.2864]), 'METEOR': 0.2151, 'ROUGE': 0.4628, 'CIDEr': 2.3378}


# CCA-TS calculation V2

In [19]:
g_cls_freq = g_conf_list/13
g_cls_freq_norm = g_cls_freq/np.max(g_cls_freq)
g_cls_freq_norm = torch.tensor(g_cls_freq_norm).float()
g_temp = model_ts.graph_su_temperature.cpu() + (g_cls_freq_norm.exp()-1.0)*.1
# g_temp = g_temp.to(device)
model_ts.graph_su_temperature.i = nn.Parameter(g_temp.to(device))
    
c_cls_freq = c_conf_list/41#train_dataset.get_cls_num_list()
c_cls_freq_norm = c_cls_freq/np.max(c_cls_freq)
c_cls_freq_norm = torch.tensor(c_cls_freq_norm).float()
c_temp =  model_ts.caption_temperature.cpu() + (c_cls_freq_norm.exp()-1.0)*.1
# c_temp = c_temp.to(device)
model_ts.caption_temperature = nn.Parameter(c_temp.to(device))
    
g_loss, c_loss, g_acc, c_acc, g_logits, g_labels, c_logits, c_labels = ts_evaluation(model_ts, dict_dataloader_val, 
                                    dataloader_val, g_temp = model_ts.graph_su_temperature, c_temp = model_ts.caption_temperature)
g_temperature_ece = ece_criterion(g_logits, g_labels).item()
c_temperature_ece = ece_criterion(c_logits, c_labels).item()
print('-----------------------------------------------------------------------')
print('CCA-TS-V2: G_loss:%.3f, G_acc:%.3f, G_ece:%.5f'%(g_loss, g_acc, g_temperature_ece))
print('CCA-TS-V2: C_loss:%.3f, C_acc:%.3f, C_ece:%.5f'%(c_loss, c_acc, c_temperature_ece))

scores, g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce = evaluate_metrics(model_ts.model, device, dict_dataloader_val, text_field, g_temp = model_ts.graph_su_temperature, c_temp = model_ts.caption_temperature)
print('CCA-TS-V2: Graph SU: acc: %0.6f map: %0.6f loss: %0.6f, ece:%0.6f, sce:%0.6f, tace:%0.6f, brier:%.6f, uce:%.6f' %(g_total_acc, g_map_value, g_total_loss, g_ece, g_sce, g_tace, g_brier, g_uce))
print("CCA-TS-V2: Caption scores :", scores)

evaluation:   0%|          | 0/447 [00:00<?, ?it/s]

-----------------------------------------------------------------------
CCA-TS-V2: G_loss:0.709, G_acc:0.575, G_ece:0.46582
CCA-TS-V2: C_loss:3.283, C_acc:0.022, C_ece:0.01962


evaluation: 100%|██████████| 447/447 [01:55<00:00,  3.87it/s]


acc: 0.575366 map: 0.320200 loss: 0.491720, ece:0.216305, sce:0.045280, tace:0.046805, brier:0.643167, uce:0.266481
CCA-TS-V2: Graph SU: acc: 0.575366 map: 0.320200 loss: 0.491720, ece:0.216305, sce:0.045280, tace:0.046805, brier:0.643167, uce:0.266481
CCA-TS-V2: Caption scores : {'BLEU': array([0.3623, 0.3231, 0.3032, 0.2864]), 'METEOR': 0.2151, 'ROUGE': 0.4628, 'CIDEr': 2.3378}
